# Batch Scoring Pipeline - Ship Date Predictions for Open Deliveries

**Goal:** Generate ship date predictions for **open deliveries** (not yet shipped).

**Use Case:** Enable operations team to:
- Forecast when orders will ship from the distribution center
- Plan logistics and inventory allocation
- Set realistic customer expectations for ship dates
- Identify orders with unusually long predicted processing times

**Workflow:**
1. Load trained regression model from MLflow
2. Get **open deliveries** using DAX (deliveries without GI Date)
3. Generate predictions: days to ship + predicted ship date
4. Save predictions to Lakehouse table: `ship_date_predictions`
5. Visualize predicted ship dates
6. Enable Power BI reporting

### 🟦 1. Import Libraries

In [ ]:
# ==============================================================================
# IMPORTS & CONFIGURATION
# ==============================================================================
# WHY: Load libraries needed to:
#      1. Load the trained model from MLflow
#      2. Query semantic model for open deliveries
#      3. Generate predictions
#      4. Save results to Lakehouse
# ==============================================================================

import sempy.fabric as fabric
import pandas as pd
import numpy as np
import mlflow
from datetime import datetime, timedelta

# Configuration - UPDATE to match your environment
DATASET = "DLV Aging Columns & Measures"  # Your semantic model name
MODEL_NAME = "ship_date_predictor"        # Model from Notebook 02
TARGET_COLUMN = "DAYS_TO_SHIP"            # What we're predicting

print("✅ Configuration loaded")
print(f"   Semantic Model: {DATASET}")
print(f"   Model: {MODEL_NAME}")
print(f"   Target: {TARGET_COLUMN}")

### 🟦 2. Configuration

In [ ]:
# ==============================================================================
# LOAD TRAINED MODEL FROM MLFLOW
# ==============================================================================
# WHY: Retrieve the model we trained in Notebook 02.
#      MLflow stores model versions and metadata.
#
# Model loading process:
# 1. Search for latest version of "ship_date_predictor"
# 2. Load the model artifact
# 3. Verify it's ready for predictions
# ==============================================================================

print("Loading trained model from MLflow...")

# Load the latest production model
model_uri = f"models:/{MODEL_NAME}/latest"
model = mlflow.sklearn.load_model(model_uri)

print(f"✅ Model loaded: {MODEL_NAME}")
print(f"✅ Model type: {type(model).__name__}")
print(f"✅ Ready for predictions")

### 🟦 3. Load Trained Model from MLflow

In [ ]:
# Load model from MLflow registry
model_uri = f"models:/{MODEL_NAME}/{MODEL_VERSION}"

print(f"Loading model from: {model_uri}")
model = mlflow.sklearn.load_model(model_uri)

print(f"✅ Model loaded successfully!")
print(f"   Model type: {type(model).__name__}")

### 🟦 4. Load Open Deliveries from Semantic Model

Load **only open deliveries** (not yet shipped) that need predictions.

In [ ]:
# ==============================================================================
# LOAD OPEN DELIVERIES (TO BE SCORED)
# ==============================================================================
# WHY: These are deliveries awaiting shipment - we want to predict when they'll ship.
#
# FILTER: ISBLANK(Aging[GI Date]) AND NOT(ISBLANK(Aging[Delivery Created On]))
#         This selects deliveries that have NOT shipped yet (no GI Date).
#         We also need Delivery Created On to calculate the predicted ship date.
#
# OPPOSITE of Notebook 02: Training used closed (shipped), scoring uses open (not shipped)
# ==============================================================================

print("Loading open deliveries from semantic model...")

ws = fabric.get_workspace_id()

# DAX query for OPEN deliveries (no GI Date = not shipped yet)
dax_query = """
EVALUATE
FILTER(
    Aging,
    ISBLANK(Aging[GI Date]) &&
    NOT(ISBLANK(Aging[Delivery Created On]))
)
"""

df_open = fabric.evaluate_dax(dataset=DATASET, dax_string=dax_query, workspace=ws)

# Clean column names
df_open.columns = [col.split('[')[-1].replace(']', '') if '[' in col else col for col in df_open.columns]

print(f"✅ Loaded {len(df_open):,} open deliveries (awaiting shipment)")
print(f"✅ Columns: {df_open.shape[1]}")
df_open.head()

### 🟦 5. Prepare Features

Extract the same features used during training.

In [ ]:
# Define features (must match training notebook)
potential_features = [
    'Plant', 'Shipping Point', 'EWM Carrier Code',
    'Brand', 'Channel', 'Product Category', 'Product Type', 'Standard Or Custom',
    'STRATEGIC_ACCOUNT', 'Sold To - Key',
    'Delivery Type', 'DELIVERY_QTY', 'DELIVERY_VALUE_USD', 'Delivery Priority', 'Shipping Condition',
    'Credit Status', 'Distribution Status', 'STATUS'
]

# Add temporal features
if 'Delivery Created On' in df_open.columns:
    try:
        df_open['created_dayofweek'] = pd.to_datetime(df_open['Delivery Created On']).dt.dayofweek
        df_open['created_month'] = pd.to_datetime(df_open['Delivery Created On']).dt.month
        potential_features.extend(['created_dayofweek', 'created_month'])
    except:
        pass

# Filter to available features
feature_cols = [f for f in potential_features if f in df_open.columns]
print(f"✅ Using {len(feature_cols)} features")

### 🟦 6. Feature Engineering

Apply the same transformations used in training.

In [ ]:
# ==============================================================================
# PREPARE FEATURES (MUST MATCH TRAINING)
# ==============================================================================
# WHY: The model was trained on specific features in a specific format.
#      We must prepare the scoring data EXACTLY the same way.
#
# Critical: Use the SAME feature list and encoding as Notebook 02!
# ==============================================================================

# Same feature list as training (from Notebook 02)
# IMPORTANT: This must match exactly what was used in training
feature_cols = [
    'Plant', 'Shipping Point', 'EWM Carrier Code',
    'Brand', 'Channel', 'Product Category', 'Product Type', 'Standard Or Custom',
    'STRATEGIC_ACCOUNT', 'Sold To - Key',
    'Delivery Type', 'DELIVERY_QTY', 'DELIVERY_VALUE_USD', 'Delivery Priority', 'Shipping Condition',
    'Credit Status', 'Distribution Status', 'STATUS'
]

# Filter to features that exist in the data
available_features = [f for f in feature_cols if f in df_open.columns]

print(f"=== Feature Matching ===")
print(f"Expected features: {len(feature_cols)}")
print(f"Available features: {len(available_features)}")

if len(available_features) < len(feature_cols):
    missing = [f for f in feature_cols if f not in df_open.columns]
    print(f"⚠️ Missing features: {missing}")


### 🟦 7. Generate Predictions

In [ ]:
# ==============================================================================
# ENCODE FEATURES (SAME AS TRAINING)
# ==============================================================================
# WHY: Convert text to numbers using the SAME method as Notebook 02.
#      The model expects numeric inputs in the exact same format.
#
# Steps: Text → "Unknown" for missing → Category codes → Fill numeric NaNs
# ==============================================================================

X_open = df_open[available_features].copy()

# Encode categorical variables (text → numbers)
categorical_cols = X_open.select_dtypes(include=['object', 'string']).columns.tolist()
for col in categorical_cols:
    X_open[col] = X_open[col].fillna('Unknown')
    X_open[col] = X_open[col].astype("category").cat.codes

# Handle numeric NaNs
numeric_cols = X_open.select_dtypes(include=['number']).columns.tolist()
for col in numeric_cols:
    if X_open[col].isnull().sum() > 0:
        X_open[col] = X_open[col].fillna(X_open[col].median())

print(f"✅ Features prepared: {X_open.shape[1]} columns, {X_open.shape[0]:,} rows")


### 🟦 8. Create Output Dataset

In [ ]:
# ==============================================================================
# GENERATE PREDICTIONS
# ==============================================================================
# WHY: Use the trained model to predict how many days from creation to ship.
#
# Output: For each open delivery, we get:
# - predicted_days_to_ship: Numeric prediction (e.g., 5 = will ship in 5 days)
# - predicted_ship_date: Delivery Created On + predicted_days_to_ship
# - lead_time_bucket: Category (0-2, 3-5, 6-9, 10+ days)
# - extended_processing: Yes/No flag for unusually long processing times
# ==============================================================================

print("\n" + "="*60)
print("GENERATING SHIP DATE PREDICTIONS FOR OPEN DELIVERIES")
print("="*60)

# Make predictions
predictions = model.predict(X_open)

print(f"✅ Generated {len(predictions):,} predictions")

# Add predictions to dataframe
df_open['predicted_days_to_ship'] = predictions

# Calculate predicted ship date
# WHY: Convert days-to-ship into an actual date for business users
df_open['predicted_ship_date'] = pd.to_datetime(df_open['Delivery Created On']) + pd.to_timedelta(df_open['predicted_days_to_ship'], unit='d')

# Create lead time buckets for categorization
# WHY: Business teams can group deliveries by expected processing time
def categorize_lead_time(days):
    if days <= 2:
        return "0-2 days"
    elif days <= 5:
        return "3-5 days"
    elif days <= 9:
        return "6-9 days"
    else:
        return "10+ days"

df_open['lead_time_bucket'] = df_open['predicted_days_to_ship'].apply(categorize_lead_time)

# Flag extended processing deliveries
# WHY: Strategic accounts + predicted >7 days = need attention
df_open['extended_processing'] = (
    (df_open.get('STRATEGIC_ACCOUNT', '') == 'Yes') & 
    (df_open['predicted_days_to_ship'] > 7)
).apply(lambda x: 'Yes' if x else 'No')

print("\n=== Prediction Summary ===")
print(f"Average predicted days to ship: {predictions.mean():.2f} days")
print(f"Max predicted days to ship: {predictions.max():.2f} days")
print(f"Extended processing deliveries: {(df_open['extended_processing'] == 'Yes').sum():,}")
print("\nLead time distribution:")
print(df_open['lead_time_bucket'].value_counts().sort_index())

### 🟦 9. Save Predictions to Lakehouse

Write predictions to a Delta table for Power BI consumption.

In [ ]:
# Save to Lakehouse
spark_df = spark.createDataFrame(output_df)

print(f"💾 Writing to Lakehouse table: {OUTPUT_TABLE}")

spark_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(OUTPUT_TABLE)

print(f"✅ Saved {len(output_df):,} predictions")
print(f"   Predicted LATE: {output_df['is_late'].sum():,}")
print(f"   Predicted ON-TIME: {(~output_df['is_late'].astype(bool)).sum():,}")

### 🟦 10. Summary

In [ ]:
# ==============================================================================
# SAVE PREDICTIONS TO LAKEHOUSE
# ==============================================================================
# WHY: Save predictions to a Lakehouse table so Power BI can consume them.
#
# Table: ship_date_predictions
# Mode: Overwrite (replaces old predictions each time)
#
# Next Step: Add this table to your Power BI semantic model and build dashboards!
# ==============================================================================

print("\n=== Saving Predictions to Lakehouse ===")

# Select key columns for output
output_cols = [
    'Delivery Document',
    'Plant',
    'Brand',
    'Channel',
    'EWM Carrier Code',
    'STRATEGIC_ACCOUNT',
    'DELIVERY_QTY',
    'DELIVERY_VALUE_USD',
    'Delivery Created On',
    'predicted_days_to_ship',
    'predicted_ship_date',
    'lead_time_bucket',
    'extended_processing'
]

# Filter to columns that exist
available_output_cols = [c for c in output_cols if c in df_open.columns]
predictions_df = df_open[available_output_cols].copy()

# Add prediction timestamp
predictions_df['prediction_timestamp'] = datetime.now()

# Save to Lakehouse table
table_name = "ship_date_predictions"
spark_df = spark.createDataFrame(predictions_df)
spark_df.write.mode("overwrite").saveAsTable(table_name)

print(f"✅ Saved {len(predictions_df):,} predictions to table: {table_name}")
print(f"✅ Columns saved: {len(available_output_cols) + 1}")
print("\n💡 Next: Add this table to your Power BI semantic model!")
print("\n=== Key Columns ===")
print(f"  • predicted_days_to_ship: Days from creation to expected ship")
print(f"  • predicted_ship_date: Calculated ship date (Delivery Created On + days)")
print(f"  • lead_time_bucket: Processing time category (0-2, 3-5, 6-9, 10+)")
print(f"  • extended_processing: Flag for strategic accounts with >7 day lead times")

---

## ✅ Predictions Complete!

The `ship_date_predictions` table is now available in your Lakehouse and can be used in Power BI for reporting and dashboards.

**Key outputs:**
- `predicted_days_to_ship`: Days from creation to expected DC ship date
- `predicted_ship_date`: Actual forecasted ship date
- `lead_time_bucket`: Processing time grouping
- `extended_processing`: Flag for strategic account deliveries with >7 day lead times